# Chapter7-3

## 見出し生成モデルの実装

今回はライブドアニュースコーパスを使用し、ニュース記事から見出しを生成する

### ライブラリのインストール

In [1]:
!pip install datasets transformers[ja,torch] sentencepiece japanize-matplotlib
!pip install mecab-python3 rouge-score sacrebleu bert_score ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 50.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

### データセットのダウンロード

In [2]:
from datasets import load_dataset

# データセットの読み込み
dataset = load_dataset("llm-book/livedoor-news-corpus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for llm-book/livedoor-news-corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/livedoor-news-corpus
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major relea

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### データセットの前処理

In [3]:
from transformers import AutoTokenizer, BatchEncoding, PreTrainedTokenizer

def preprocess_data(
    data: dict[str, any], tokenizer: PreTrainedTokenizer
) -> BatchEncoding:
  """ データの前処理 """

  # 記事のトークナイゼーション
  inputs = tokenizer(
      data["content"], max_length=512, truncation=True
  )

  # 見出しのトークナイゼーション
  # 見出しはトークンIDのみ使用
  inputs["labels"] = tokenizer(
      data["title"], max_length=128, truncation=True
  )["input_ids"]

  return inputs

# トークナイザーの読み込み
model_name = "retrieva-jp/t5-base-long"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 学習セットに対して前処理
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=dataset["train"].column_names,
)

# 検証セットに対して前処理
val_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=dataset["validation"].column_names,
)

tokenizer_config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/5893 [00:00<?, ? examples/s]

Map:   0%|          | 0/736 [00:00<?, ? examples/s]

In [4]:
# トークナイゼーションした学習データセットの中身を出力
print(train_dataset["labels"])

[[8, 1653, 2394, 20260, 63, 15904, 15, 29051, 3321, 63, 4005, 1], [8, 17280, 25976, 2350, 2418, 720, 77, 2439, 8033, 6, 9713, 14505, 5584, 11592, 14352, 17763, 10110, 15955, 473, 140, 1], [8, 18632, 6390, 35, 77, 2634, 30234, 8, 17912, 26256, 1], [8, 23, 2799, 2117, 9397, 24, 14, 7825, 911, 8, 12119, 8, 16725, 7, 6935, 666, 1101, 49, 884, 14, 8362, 4694, 62, 1856, 9848, 265, 3240, 140, 1], [8, 24610, 24573, 77, 8, 947, 2462, 23, 369, 24828, 579, 8, 24050, 10, 19622, 101, 24, 315, 265, 5258, 140, 1], [8, 3794, 5093, 3, 26567, 8, 4701, 4, 25473, 1161, 987, 383, 23, 26567, 8, 4701, 1191, 2496, 100, 24, 3159, 1], [988, 1319, 1701, 140, 12368, 1970, 10729, 10631, 4, 1645, 17624, 3281, 18, 1064, 345, 3029, 1269, 4902, 597, 3, 121, 1944, 4108, 1152, 26, 27, 16, 4418, 1], [1180, 250, 165, 602, 14, 21805, 77, 8, 1373, 3101, 2324, 23, 28416, 8, 27427, 90, 8, 23209, 24, 1656, 1], [8, 1128, 21, 7, 10562, 15, 17936, 4, 27, 77, 8, 22648, 4, 1014, 25019, 18, 3, 3009, 20726, 6056, 77, 1], [8, 5298, 94

### モデルのファインチューニング

今回は系列変換に対応した`Seq2SeqTrainer`を用いてファインチューニングをする

In [5]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

# モデルの読み込み
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# collate関数の定義
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.trainer_utils import set_seed

# 乱数シードを固定
set_seed(42)

# Trainerに渡す引数を初期化
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Learning_LLM/chapter7/train",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

# Trainenerの初期化
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# 学習開始
trainer.train()
trainer.save_model("/content/drive/MyDrive/Learning_LLM/chapter7/model")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.889400,2.000472
2,2.291800,1.897983
3,2.040300,1.868544
4,1.898000,1.856829


Epoch,Training Loss,Validation Loss
1,3.889400,2.000472
2,2.291800,1.897983
3,2.040300,1.868544
4,1.898000,1.856829
5,1.812000,1.857560


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


### モデルの評価

In [7]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, BatchEncoding, PreTrainedTokenizer
from transformers.trainer_utils import set_seed

In [6]:
# モデルの読み込み (ファインチューニングしたモデルは必要ない)
model_name = "llm-book/t5-base-long-livedoor-news-corpus"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

### 見出しの生成

In [ ]:
from torch.utils.data import DataLoader
from transformers import PreTrainedModel
from tqdm import tqdm

def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:

  """  ミニバッチのデータを事例単位のlistに変換 """
  dict_list = []

  # dictのkeyのリストを作成
  keys = list(list_dict.keys())
  for idx in range(len(list_dict[keys[0]])):
    dict_list.append({key: list_dict[key][idx] for key in keys})

  return dict_list

def run_generation(
    dataloader: DataLoader, model: PreTrainedModel
) -> list[dict[str, any]]:
  """ 見出し生成 """

  generations = []
  for batch in tqdm(dataloader):

    del batch["labels"]

    batch = {k: v.to(model.device) for k, v in batch.items()}

    # 見出しのトークンIDを設定
    batch["generated_title_ids"] = model.generate(**batch)
    batch = {k: v.cpu().tolist() for k, v in batch.items()}

    # ミニバッチのデータを事例単位のリストに変換
    generations += convert_list_dict_to_dict_list(batch)

  return generations

In [ ]:
# テストセットに対して前処理
test_dataset = dataset["test"].map(
    preprocess_data,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=dataset["test"].column_names
)

# 3つ目の事例を表示
print(test_dataset[2])

{'input_ids': [8, 5863, 5863, 2392, 8313, 4, 3765, 15, 23282, 4, 14, 597, 32, 3029, 106, 934, 597, 5, 206, 3, 2128, 14, 29598, 3258, 22, 3, 5368, 93, 17279, 17279, 3765, 15, 27564, 386, 3710, 28145, 5, 5368, 15, 4011, 320, 8313, 4, 7185, 7425, 3765, 20176, 53, 3, 2350, 958, 10089, 16, 22410, 173, 1217, 5765, 1011, 13156, 115, 5, 8, 480, 3, 565, 874, 772, 44, 597, 2350, 958, 10089, 16, 3, 14997, 12400, 26943, 597, 71, 9628, 4872, 23, 1973, 605, 13324, 3, 3548, 736, 5, 24, 4120, 26575, 2487, 5, 11298, 12034, 169, 7691, 173, 16631, 602, 4, 2199, 1944, 368, 16, 4312, 18, 9949, 32, 3, 1336, 30, 4120, 660, 9180, 5, 4120, 660, 44, 3, 565, 874, 772, 7882, 3926, 14, 6113, 6014, 23, 1973, 605, 13324, 24313, 810, 24, 14, 3508, 1133, 24788, 4414, 4418, 35, 57, 21, 273, 752, 2157, 82, 115, 5, 8, 61, 12034, 169, 7691, 173, 8312, 785, 39, 1336, 193, 121, 27164, 4, 23, 21693, 69, 2244, 8, 25704, 8, 884, 2806, 3148, 24, 7, 3, 28, 4528, 23, 32, 3029, 106, 28681, 25704, 24, 30, 3, 16262, 16, 5180, 173, 2

In [ ]:
# 見出しトークンの設定は入力があればいいのでラベルを削除
test_dataset = test_dataset.remove_columns(["labels"])

# 3つ目の事例を表示
print(test_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 738
})


In [ ]:
test_dataloader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator
)

dir(test_dataloader)


['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [ ]:
for batch in test_dataloader:
  del batch["labels"]

  print(batch)

{'input_ids': tensor([[    8,    23,  7453,  ...,     0,     0,     0],
        [    8,  1384,  4699,  ...,     0,     0,     0],
        [    8,  5863,  5863,  ...,  1197,    22,     1],
        ...,
        [    8,  4354,    99,  ...,     0,     0,     0],
        [    8, 15017,  6570,  ...,     0,     0,     0],
        [    8,  1677,  6570,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[    8,  3628,    18,  ...,    49,   191,     1],
        [    8, 16941,     3,  ...,     0,     0,     0],
        [    8,  4341,  1216,  ..., 14405,  2559,     1],
        ...,
        [    8, 19602,  7932,  ...,     0,     0,     0],
        [ 1666, 21760,    23,  ...,     0,     0,     0],
        [    8,   558,  8156,  ...,  9913,     7,     1]]), 'attentio

In [ ]:
# 見出し語の生成
generations = run_generation(test_dataloader, model)

  0%|          | 0/93 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 93/93 [32:43<00:00, 21.11s/it]


In [ ]:
# 生成した見出しを出力
tokens = tokenizer.convert_ids_to_tokens(
    generations[0]["generated_title_ids"]
)

print(tokens)

['<pad>', '▁', '今日は', 'そういう', '日', 'だった', 'のか', '!', 'Google', 'ロゴ', 'が', '変わって', 'いる', '理由', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
def postprocess_title(
    generations: list[dict[str, any]],
    dataset: list[dict[str, any]],
    tokenizer: PreTrainedTokenizer
):
  """ 見出しの後処理 """

  results = []
  for generation, data in zip(generations, dataset):
    # IDのリストをテキストに変換
    data["generated_title"] = tokenizer.decode(
        generation["generated_title_ids"],
        skip_special_tokens=True
    )

    results.append(data)

  return results



In [ ]:
# 見出しテキストを生成
results = postprocess_title(generations, dataset["test"], tokenizer)
print(results[0]["generated_title"])

今日はそういう日だったのか!Googleロゴが変わっている理由


### モデルを評価

In [ ]:
import ipadic
import MeCab

# 単語分割を行う
tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")

In [ ]:
from collections import defaultdict
import pandas as pd
from datasets import load_metric

# 小数点以下の桁数を3に設定
pd.options.display.precision = 3

def convert_words_to_ids(
    predictions: list[str], reference: list[str]
) -> tuple[list[str], list[str]]:
  """ 単語列をID列に変換 """

  # 単語にユニークなIDを割り当てるためのdefaultdictを作成する
  word2id = defaultdict(lambda: len(word2id))

  # 単語区切りの文字列をID文字列に変換
  pred_ids = [
      " ".join([str(word2id[w]) for w in p.split()])
      for p in predictions
  ]

  ref_ids = [
      " ".join([str(word2id[w]) for w in r.split()])
      for r in reference
  ]

  return pred_ids, ref_ids

# 評価を算出する関数をそれぞれ定義
def compute_rouge(
    predictions: list[str], reference: list[str]
) -> dict[str, dict[str, float]]:
  """ ROUGEを算出 """

  # ROUGEをロード
  rouge = load_metric("rouge")

  # 単語列をID列に変換
  pred_ids, ref_ids = convert_words_to_ids(predictions, reference)

  # 単語ID列を評価対象に加える
  rouge.add_batch(predictions=pred_ids, references=ref_ids)

  # スコアの計算
  scores = rouge.compute(rouge_types=["rouge1", "rouge2", "rougeL"])

  return {k: v.mid for k, v in scores.items()}

def compute_bleu(
    predictions: list[str], reference: list[str]
) -> dict[str, int | float | list[float]]:
  """ BLEUを算出 """

  # sacreBLEUをロード
  bleu = load_metric("sacrebleu")

  # 単語列を評価対象に加える
  bleu.add_batch(predictions=predictions, references=reference)

  # BLEUを計算
  results = bleu.compute()
  results["precisions"] = [
      round(p, 2) for p in results["precisions"]
  ]

  return results

def compute_bertscore(
    predictions: list[str], references: list[str]
) -> dict[str, float]:
  """ BERTScoreの算出 """

  # BERTScoreのロード
  bertscore = load_metric("bertscore")

  # 評価対象に追加
  bertscore.add_batch(
      predictions=predictions, references=references
  )

  # BERTScoreを計算
  results = bertscore.compute(lang="ja")

  return {
      k: sum(v) / len(v)
      for k, v in results.items()
      if k != "hashcode"
  }

In [ ]:
# ROUGEを算出して表示
generated_titles = [
    tagger.parse(r["generated_title"]).strip() for r in results
]

# 正解データ
ref_titles = [tagger.parse(r["title"]).strip() for r in results]

# 算出して結果を表示
rouge_results = compute_rouge(generated_titles, ref_titles)
display(pd.DataFrame.from_dict(rouge_results, orient="index"))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


,precision,recall,fmeasure
rouge1,0.417,0.319,0.352
rouge2,0.214,0.156,0.176
rougeL,0.365,0.278,0.306


ある程度高いスコアを達成している

`precision`が`recall`の値より高くなっている

$\rightarrow$生成した見出しが参照する見出しよりも短くなっている傾向がある

In [ ]:
# BLUEを算出

ref_titles = [[tagger.parse(r["title"]).strip()] for r in results]

# 算出して結果を表示
bleu_results = compute_bleu(generated_titles, ref_titles)
display(pd.DataFrame([bleu_results]).rename(index={0: "BLEU"}).T)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


,BLEU
score,13.523
counts,"[4439, 2151, 1292, 804]"
totals,"[10611, 9873, 9135, 8397]"
precisions,"[41.83, 21.79, 14.14, 9.57]"
bp,0.721
sys_len,10611
ref_len,14075


生成した見出しの文が、参照する見出しよりも短く、BPで示されるペナルティによる影響が大きい可能性がある

In [ ]:
# BERTScoreで算出

generated_titles = [r["generated_title"].strip() for r in results]
ref_titles = [r["title"].strip() for r in results]

# 計算して結果を表示
bertscore_results = compute_bertscore(generated_titles, ref_titles)
display(pd.DataFrame([bertscore_results]).rename(index={0: "BERTScore"}))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

,precision,recall,f1
BERTScore,0.757,0.726,0.741


In [ ]:
# 記事、見出し、生成した見出しを表示
display(
    pd.DataFrame(results)[:3][["content", "title", "generated_title"]]
)

,content,title,generated_title
0,「今日はそういう日だったのか！Googleロゴが変わっている理由」で紹介したように、Goog...,なるほど、そういうことか！Googleロゴが折り紙である理由,今日はそういう日だったのか!Googleロゴが変わっている理由
1,ビデオサロン12月号の記事連動の動画です。今回のテーマは「音量の合わせ方」です。詳しくは20...,【記事連動】音の編集講座「音量の合わせ方」【ビデオSALON】,音量の合わせ方【ビデオSALON】
2,みずみずしい理想の肌に欠かせないのが”しっとり感”。しかし、冬が近づいてくると、乾燥によるカ...,【終了しました】しっとりなめらかな美肌を作る「ちふれ ベースメイクセット」を3名様にプレゼント,乾燥知らずのうるおい美肌を手に入れる「ちふれ ベースメイクセット」


生成された文は見出しの文体になっている。

全体的に記事の重要な内容を生成することができている

## 多様な生成方法による見出し生成

ここではtransformer上で実行されており、ハイパーパラメータの
変更のみでできるテキスト生成方法を実施

In [10]:
content = dataset["test"][434]["content"]
title = dataset["test"][434]["title"]

# 確認用
print(f"content: {content}")
print(f"title: {title}")

content: そろそろ梅雨がやってきます。この時期に多い女子のお悩みといえば、ヘアスタイルに関すること。  湿気で広がった髪がどうしてもまとまらず、とりあえず適当にまとめて家を飛び出したり、帽子を被ってごまかしたり……という経験、ありますよね。でも、じめじめした季節だってかわいく・オシャレに過ごしたいもの。  髪が広がりやすい梅雨の時期は、アレンジヘアで乗り切るのがベスト。「ヘアアレンジというと、難しそうな印象がありますが、ちょっとしたコツさえおさえれば、ぱぱっと簡単にできますよ！」と話してくれたのは、美容室「MADURiCA por DIFINO」のスタイリスト、山口祐亮さん。  ではさっそく、バクハツしがちなヘアをオシャレにごまかすアレンジテクニックを教えてください！  バクハツヘアもすっきり！ 楽ちんヘアアレンジ  元の髪はこんな感じ。ここからアレンジをしていきます。 基本の「まとまる」テクニック おしゃれなアレンジヘアに見せるポイントは、アレンジの前に毛先をアイロンやカーラーで巻いておくこと。このひと手間でまとまり方や仕上がりが変わります！  毛先に動きがあるだけで、ただのポニーテールもぐっとオシャレに見えるし、後れ毛だってアクセントに。コツは髪をいくつもの部分に分けて少しずつ巻くこと。慣れてしまえばそんなに時間はかかりません。  ＜簡単アレンジ1＞ ふんわりルーズ感がかわいい！ 簡単アップスタイル  完成形 蒸し暑い日はアップスタイルにしたいですよね。 このアレンジなら、編み込みがアクセントになって横顔美人！ 後ろから見たときも、ランダムに散らばった毛先がキュートです。下準備として、毛先はアイロンやカーラーで巻いておきましょう。   1.  前髪や顔周りの髪は残し、トップ〜耳上までの髪を前から見て約4：6くらいの割合になるように分け、多い方をざっくりと編み込みにして、ゴムで止めます。  編み込みを作ったら、少しずつ毛を引き出してルーズな感じにするとかわいい印象に。   2.  髪の毛の量が多い人は、後ろ髪を上下に分けて、下側の真ん中あたりの髪の一部を三つ編みにしてねじり、ピンで留めます。  これが土台になり、このあとピンをさしやすくなります。三つ編みにする量は自分の髪の量に合わせて調節してください。   3.  残った髪を何束かに分け、ねじって丸めなが

In [11]:
from functools import partial
from transformers import pipeline

# 乱数シードを固定
set_seed(42)

# モデルを固定したpipelineを作成
fixed_model_pipeline = partial(
    pipeline,
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device="cuda"
)

In [ ]:
# 確認用
fixed_model_pipeline

functools.partial(<function pipeline at 0x78d7d0ad2c20>, 'summarization', model=T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [12]:
# 貪欲法で見出しを生成
print(fixed_model_pipeline()(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨のヘアスタイルは簡単アレンジで! 簡単アレンジで梅雨を乗り切りよう


In [13]:
# nグラムによるペナルティを導入して生成する (同じトークンが生成されるのを防ぐ)
summarization_pipeline = fixed_model_pipeline(no_repeat_ngram_size=2)
print(summarization_pipeline(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨のヘアスタイルは簡単アレンジで! 簡単ヘアアレンジテクニック【ビューティー特集】


In [15]:
# ビームサーチを使った見出しの生成
summarization_pipeline = fixed_model_pipeline(num_beams=3)
print(summarization_pipeline(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切ろう!


In [16]:
# 複数の候補の見出しを表示
summarization_pipeline = fixed_model_pipeline(
    num_beams=3, num_return_sequences=3
)

for summary in summarization_pipeline(content):
  print(summary["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切ろう!
梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切ろう
梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切りよう!


In [18]:
# サンプリングを利用した見出しの生成
summarization_pipe = fixed_model_pipeline(do_sample=True, top_k=0)
print(summarization_pipe(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


うっすらパーマで崩れにくい!? 梅雨入りを涼しくパチリ!


In [19]:
# 温度付きソフトマックス関数を使った見出しの生成
# 温度を低く
summarization_pipe = fixed_model_pipeline(
    do_sample=True, top_k=0, temperature=0.5
)

print(summarization_pipe(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨のヘアアレンジは簡単! 簡単アレンジで梅雨を乗り切る【ビューティー特集


In [20]:
# 温度付きソフトマックス関数を使った見出しの生成
# 温度を高く
summarization_pipe = fixed_model_pipeline(
    do_sample=True, top_k=0, temperature=1.3
)

print(summarization_pipe(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨の自然といえば〜!?セルフアレンジでボディメイク正解トレンドチェック【レポート】


In [22]:
# top_kサンプリングを利用した見出し生成
summarization_pipeline = fixed_model_pipeline(
    do_sample=True, top_k=10, temperature=1.3
)

print(summarization_pipeline(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨はヘアスタイルで乗り切りましょう!梅雨のヘアスタイルはアレンジに気をつけたい! -


In [23]:
# top_k, top_pを用いたサンプリング
summarization_pipe = fixed_model_pipeline(
    do_sample=True, top_k=0, top_p=0.5, temperature=1.3
)

print(summarization_pipe(content)[0]["summary_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


梅雨時にこそ必要! 大人女子のための簡単アレンジ術


### 長さを調整してテキストを生成

トークン数は出力した時の文字の長さを調整することができるわけではない

In [24]:
summarization_pipeline = fixed_model_pipeline(
    num_beams=3,
    num_return_sequences=3,
    min_new_tokens=5, # 最大トークン数
    max_new_tokens=5, # 最小トークン数
)

for summary in summarization_pipeline(content):
  print(summary["summary_text"])

梅雨のヘアスタイルは
梅雨のヘアスタイルを
梅雨のヘアアレンジ


In [25]:
summarization_pipeline = fixed_model_pipeline(
    num_beams=3,
    num_return_sequences=3,
    min_new_tokens=35, # 最大トークン数
    max_new_tokens=35, # 最小トークン数
)

for summary in summarization_pipeline(content):
  print(summary["summary_text"])

梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切ろう【オトナ女子のリアルな悩み解決術 vol.7】Presented
梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切ろう【オトナ女子のリアルな悩み解決術 vol.7】【ビューティー特集】
梅雨のヘアスタイルをオシャレに! 簡単アレンジで梅雨を乗り切ろう【オトナ女子のリアルな悩み解決術 vol.7】【ビューティー特集】|


In [27]:
# サンプリングを利用し、指定した長さの見出しを出力
summarization_pipeline = fixed_model_pipeline(
    num_beams=3,
    num_return_sequences=3,
    min_new_tokens=35, # 最大トークン数
    max_new_tokens=35, # 最小トークン数
    do_sample=True,
    temperature=1.3,
    no_repeat_ngram_size=3,
)

for summary in summarization_pipeline(content):
  print(summary["summary_text"])

梅雨のヘアスタイルは簡単アレンジで! 女子力アップのアレンジテクニック【ビューティー特集vol.5】【ヘアアレンジ特集】【ビューティー特集】【ヘアメイク特集】【ファッション
梅雨のヘアスタイルは簡単アレンジで! 女子力アップのアレンジテクニック【ビューティー特集vol.5】【ヘアアレンジ特集】【ビューティー特集】【ヘアメイク特集】【メイク
梅雨のヘアスタイルは簡単アレンジで! 女子力アップのアレンジテクニック【ビューティー特集vol.5】【ヘアアレンジ特集】【ビューティー特集】【ヘアメイク特集】【美容
